# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver
- Chrome: chromedriver
- Firefox: geckodriver

pyautogui = automatizar mouse, tela e teclado
- são automacoes que impedem de usar o pc
- funciona em qualquer ferramenta, sistema e programa
- pyautogui se tela mudar, ele para o programa

selenium = automatizar web/internet
- só funciona para a internet
- funciona mesmo com o usuario fazendo outra atividade no pc
- funciona mesmo se a posicao das coisas na tela mudar


In [108]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#abrir o navegador
navegador = webdriver.Chrome()

In [109]:
#pesquisar cotacao dolar
navegador.get("https://www.google.com.br/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar valor da busca no google
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

5.438336


In [110]:
#pesquisar cotacao euro
navegador.get("https://www.google.com.br/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar valor da busca no google
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

6.53685268


In [111]:
#pesquisar cotacao do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

#pegar valor do ouro
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

309.14


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [112]:
import pandas as pd

tabela_produtos = pd.read_excel("Produtos.xlsx")
#transformar para float colunas que estao em object
#tabela_produtos ["Preço Final"] = pd.to_numeric(tabela_produtos["Preço Final"], errors="coerce") -- converter de string para numeric
#tabela_produtos['Moeda'] = tabela_produtos['Moeda'].astype(str)   ---- converter de numeric para string

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [113]:
#passo 1: atualizar cotacoes
tabela_produtos.loc[tabela_produtos["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos["Moeda"]=="Euro", "Cotação"] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos["Moeda"]=="Ouro", "Cotação"] = float(cotacao_ouro)

#passo 2: atualizar base reais = cotacao * base original
tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']

#passo 3: atualizar preço final = base reais * ajuste
#formatar os dados
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map("{:.2f}".format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.438336,5438.281617,1.40,7613.59
1,Carro Renault,4500.00,Euro,6.536853,29415.837060,2.00,58831.67
2,Notebook Dell,899.99,Dólar,5.438336,4894.448017,1.70,8320.56
3,IPhone,799.00,Dólar,5.438336,4345.230464,1.70,7386.89
4,Carro Fiat,3000.00,Euro,6.536853,19610.558040,1.90,37260.06
5,Celular Xiaomi,480.48,Dólar,5.438336,2613.011681,2.00,5226.02
6,Joia 20g,20.00,Ouro,309.140000,6182.800000,1.15,7110.22


### Agora vamos exportar a nova base de preços atualizada

In [114]:
tabela_produtos.to_excel("Produtos Atualizados.xlsx", index=False)